# Interactive Tutorial for the HWO UV Instrument (UVI) 

[SYOTools](https://github.com/spacetelescope/hwo-tools) is a framework to enable users to create [Science Yield Optimization web tools](http://hwo.stsci.edu) for observatory design. It uses Bokeh & astropy to visualize exposure time calculators (ETCs), as well as other science cases, as a function of various observatory parameters. While SYOTools can be used independently of a particular observatory or science case, it has been primarily written to facilitate the design of [HWO](https://www.habitableworldsobservatory.org).

SYOTools is divided into two main parts, contained in the `syotools.models` and `syotools.interface` subpackages. This Jupyter Notebook is intended as a walkthrough for using `syotools.models` to perform calculations. A future tutorial on using `syotools.interface` to design a web tool is in preparation; however, it will not be a similar Jupyter Notebook, as the interface framework is highly integrated with Bokeh Server. Instead, we will use Bokeh's `output_notebook` function, along with IPython interactors (as described [here](https://github.com/bokeh/bokeh/blob/0.12.13/examples/howto/notebook_comms/Jupyter%20Interactors.ipynb)), to visualize the example calculations below. 

In [1]:
#Import Bokeh interface tools
from ipywidgets import interact
from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
from bokeh.layouts import column
output_notebook()

#Import numpy, syotools, and astropy.units
### NOTE: you will likely receive some pysynphot warnings when executing these imports, these are expected
import numpy as np
from syotools import cdbs #make sure that pysynphot can find its reference files!
from syotools.models import Telescope, Camera, Spectrograph #models for the observatory and instruments
from syotools.utils.jsonunit import str_jsunit #for printing JsonUnit and JsonSpectrum wrappers in a readable way
from syotools.utils import pre_encode, pre_decode
from syotools.spectra import SpectralLibrary
import astropy.units as u

Loading BokehJS ...

Warning, 5 of 20000 bins contained negative fluxes; they have been set to zero.


/Users/tumlinson/anaconda3/envs/hwotools/lib/python3.9/site-packages/pysynphot/units.py:824: RuntimeWarning: divide by zero encountered in log
  return -1.085736 * N.log(arg) + ABZERO


Warning, 3590 of 20000 bins contained negative fluxes; they have been set to zero.
Warning, 293 of 2920 bins contained negative fluxes; they have been set to zero.


## Example 2: UVI ETC

*"The UV Instrument (HWO UVI) is a multi-object spectrograph covering far-ultraviolet (100 nm) through visible (850 nm) wavelengths. UVI is a highly multiplexed ultraviolet spectrograph, with medium and low-resolution multi-object imaging spectroscopy and FUV imaging modes. UVI can be considered as a successor to the Hubble Space Telescope Imaging Spectrograph (STIS) instrument, with two orders-of-magnitude higher efficiency, multi-object capability, and a wide-field multi-band imaging channel."* Adapted from the LUVOIR STDT study of LUMOS

In this example, we will create a exposure time calculator for UVI, so that we can calculate the signal-to-noise ratio (SNR) for several possible template spectra. This approximates some of the functionality of [the official UVI ETC tool](http://hwo.stsci.edu/uvipec_etc).

In [12]:
#Again, instatiate the observatory using syotools.models.Telescope
hwo_ex1 = Telescope()
hwo_ex1.set_from_json('EAC1') 

Setting Telescope to:  EAC1


In [13]:
#Instantiate the instrument, using syotools.models.Spectrograph, and link it with the telescope
#This loads the default Spectrograph values, which are already based on the HWO-UVI design
uvi = Spectrograph()
hwo_ex2.add_spectrograph(uvi)

In [14]:
#Create a new spectrographic exposure for the camera
uvi_exposure = uvi.create_exposure()

In [15]:
#Create a new spectrographic exposure for the camera
uvi_exposure = uvi.create_exposure()

uvi_exposure.sed_id = "qso"
uvi_exposure.renorm_sed(21.0 * u.ABmag, bandpass='galex,fuv')

#Print the available grating modes
available_uvi_modes = [uvi.descriptions[mode] for mode in uvi.modes]
print("UVI grating modes:") 
for mode in available_uvi_modes:
    print("   "+mode) 

#Print the default template & mode
default_uvi_template = SpectralLibrary[uvi_exposure.sed_id]
default_uvi_mode = uvi.descriptions[uvi.mode]
print("Current SED template: {}".format(default_uvi_template)) 
print("Current grating mode: {}".format(default_uvi_mode)) 

uvi_template_codes = ['flam', 'qso', 's99', 'o5v', 'g2v', 'g191b2b', 'gd71', 'gd153', 'ctts', 
                        'mdwarf', 'orion', 'nodust', 'ebv6', 'hi1hei1', 'hi0hei1']
available_uvi_templates = [SpectralLibrary[tc] for tc in uvi_template_codes]

UVI grating modes:
   G120M (R = 30,400)
   G150M (R = 37,800)
   G180M (R = 40,800)
   G155L (R = 11,600)
   G145LL (R = 500)
   G300M (R = 28,000)
Current SED template: QSO
Current grating mode: G150M (R = 37,800)


In [16]:
#Create the Bokeh SED figure
uvi_sed = uvi_exposure.recover('sed')
uvi_sed.convert('flam')
uvi_sed.convert('angstrom')
uvi_bgflux = uvi.recover('bef').to(u.erg / u.s / u.cm**2 / u.AA).value
uvi_bgwave = uvi.recover('wave').to(u.AA).value
uvi_sed_fig = figure(height=300, width=600, title="SED", x_axis_label="Wavelength [Angstrom]",
                 y_axis_label="Flux [erg / s / cm2 / Ang]", y_range=(0., 4.0e-16), x_range=(900, 4000))
uvi_sed_line = uvi_sed_fig.line(uvi_sed.wave, uvi_sed.flux, color='firebrick', 
                                    line_width=3) #legend='Source Flux')
uvi_bef_line = uvi_sed_fig.line(uvi_bgwave, uvi_bgflux, color='darksalmon', 
                                    line_width=3) #  legend='Background')

In [17]:
#Create the Bokeh SNR figure
uvi_snr = uvi_exposure.recover('snr')
uvi_snr_fig = figure(height=300, width=600, title="SNR", x_axis_label="Wavelength [Angstrom]", 
                 y_axis_label="S/N per resel", y_range=(0.0, 40.0), x_range=(900, 4000))
uvi_snr_line = uvi_snr_fig.line(uvi_bgwave, uvi_snr, color='orange', line_width=3)

In [18]:
#Define the update callback function for interactive inputs
def uvi_update(template=default_uvi_template, aperture=15., exptime=1., 
                 v_magnitude=21., mode=default_uvi_mode):
    #find the correct template code & grating mode
    sed_id = uvi_template_codes[available_uvi_templates.index(template)]
    grating, = [m for m, d in uvi.descriptions.items() if d == mode]
    
    #turn off calculations until everything is updated
    uvi_exposure.disable()
    
    #update all of the telescope, spectrograph, & exposure parameters
    hwo_ex2.aperture = aperture * u.m
    uvi.mode = grating
    uvi_exposure.exptime = exptime * u.h
    uvi_exposure.sed_id = sed_id
    uvi_exposure.renorm_sed(v_magnitude * u.ABmag, bandpass='galex,fuv')
    
    #turn calculations back on and recalculate based on updated parameters
    uvi_exposure.enable()
    
    #recover the recalculated values, and make sure everything is in the right units
    uvi_sed, uvi_snr = uvi_exposure.recover('sed', 'snr')
    uvi_sed.convert('flam')
    uvi_sed.convert('angstrom')
    uvi_bgflux = uvi.recover('bef').to(u.erg / u.s / u.cm**2 / u.AA).value
    uvi_bgwave = uvi.recover('wave').to(u.AA).value
    
    #sanitize the fluxes and SNR because some of the spectra don't play nice
    uvi_flux = uvi_sed.flux
    uvi_flux[~np.isfinite(uvi_flux)] = 0.
    uvi_bgflux[~np.isfinite(uvi_bgflux)] = 0.
    uvi_snr[~np.isfinite(uvi_snr)] = 0.
    
    #update the SED figure
    uvi_sed_fig.y_range.start = 0.
    uvi_sed_fig.y_range.end = 1.5 * uvi_flux.max()
    uvi_sed_line.data_source.data = {'x': uvi_sed.wave, 'y': uvi_flux}
    uvi_bef_line.data_source.data = {'x': uvi_bgwave, 'y': uvi_bgflux}
    
    #update the SNR figure
    uvi_snr_fig.y_range.start = 0.
    uvi_snr_fig.y_range.end = 1.5 * uvi_snr.value.max()
    uvi_snr_line.data_source.data = {'x': uvi_bgwave, 'y': uvi_snr.value}
    
    #update the plots
    push_notebook()

In [22]:
#Show the plots
# depending on your jupyter config you may need to shift+enter this cell 
# after changing the sliders in the cell below to see an updated result plot
uvi_handle = show(column(uvi_sed_fig, uvi_snr_fig), notebook_handle=True)

In [23]:
#Create the interactive inputs
#shift+enter here will reset to defaults. 
uvi_inputs = interact(uvi_update, template=available_uvi_templates, aperture=(2.0, 12.0), 
                        exptime=(0.1, 10.0, 0.1), v_magnitude=(15.0, 30.0, 0.1), mode=available_uvi_modes)

interactive(children=(Dropdown(description='template', index=1, options=('Flat in F_lambda', 'QSO', '10 Myr St…